In [18]:
!pip install google-cloud-storage google-cloud-bigquery


Defaulting to user installation because normal site-packages is not writeable


In [21]:
from google.cloud import storage, bigquery
import os

---
# Étape 1 : Configuration des ressources GCP

In [57]:
# Configuration de base
PROJECT_ID = "isi-group-m2-dsia"
BUCKET_NAME = "m2dsia-ndao-ibrahima-data"
DATASET_NAME = 'dataset_ndao_ibrahima'
TABLE_NAME = 'table_ndao_ibrahima'


In [48]:
client = storage.Client(project=PROJECT_ID)
bucket = client.get_bucket(BUCKET_NAME)

In [49]:
bucket

<Bucket: m2dsia-ndao-ibrahima-data>

In [50]:
def create_folder_bucket(bucket_name):
    # Define folder structure
    folders = ["input/", "clean/", "error/", "done/"]
    for folder in folders:
        blob = bucket.blob(folder)
        blob.upload_from_string('')  # Create an empty folder
        print(f"Folder {folder} created in bucket {bucket_name}.")

In [51]:
create_folder_bucket(BUCKET_NAME)

Folder input/ created in bucket m2dsia-ndao-ibrahima-data.
Folder clean/ created in bucket m2dsia-ndao-ibrahima-data.
Folder error/ created in bucket m2dsia-ndao-ibrahima-data.
Folder done/ created in bucket m2dsia-ndao-ibrahima-data.


In [61]:
blobs = client.list_blobs(BUCKET_NAME)
for blob in blobs:
    print(blob.name)

clean/
done/
error/
input/


In [58]:
def create_bigquery_table(project_id, dataset_name, table_name):
    """Create a BigQuery dataset and table."""
    client = bigquery.Client()

    # Create dataset
    dataset_id = f"{project_id}.{dataset_name}"
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = "africa-south1"  # Adjust region as needed
    dataset = client.create_dataset(dataset, exists_ok=True)
    print(f"Dataset {dataset_id} created or already exists.")

    # Define table schema
    schema = [
        bigquery.SchemaField("transaction_id", "INT64", mode="REQUIRED", description="Identifiant unique"),
        bigquery.SchemaField("product_name", "STRING", mode="REQUIRED", description="Nom du produit"),
        bigquery.SchemaField("category", "STRING", mode="REQUIRED", description="Catégorie du produit"),
        bigquery.SchemaField("price", "FLOAT64", mode="REQUIRED", description="Prix unitaire"),
        bigquery.SchemaField("quantity", "INT64", mode="REQUIRED", description="Quantité achetée"),
        bigquery.SchemaField("date", "DATE", mode="REQUIRED", description="Date de la transaction"),
        bigquery.SchemaField("customer_name", "STRING", mode="NULLABLE", description="Nom du client"),
        bigquery.SchemaField("customer_email", "STRING", mode="NULLABLE", description="E-mail du client"),
    ]

    table_id = f"{dataset_id}.{table_name}"
    table = bigquery.Table(table_id, schema=schema)

    # Set partitioning and clustering
    table.time_partitioning = bigquery.TimePartitioning(
        type_=bigquery.TimePartitioningType.DAY,
        field="date"  # Specify the partitioning field
    )
    table.clustering_fields = ["category", "product_name"]

    # Create the table
    table = client.create_table(table, exists_ok=True)
    print(f"Table {table_id} created or already exists.")

In [59]:
create_bigquery_table(PROJECT_ID, DATASET_NAME, TABLE_NAME)

Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/isi-group-m2-dsia/datasets?prettyPrint=false: Access Denied: Project isi-group-m2-dsia: User does not have bigquery.datasets.create permission in project isi-group-m2-dsia.